In [1]:
import keras
import tensorflow as tf
import numpy as np
from mobilenet import MobileNet
import sys
sys.path.append('../CustomLayers/')
sys.path.append('../Keras/')
from TB_writer import TB_writer
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input

Using TensorFlow backend.


In [2]:
train_datagen = ImageDataGenerator(
        rescale = 1./255,
        shear_range = 0.2,
        zoom_range = 0.2,
        horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [3]:
train_generator = train_datagen.flow_from_directory(
        '/data/imagenet/train/',
        target_size = (224, 224),
        batch_size=128,
        class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
        '/data/imagenet/val/',
        target_size = (224, 224),
        batch_size=128,
        class_mode='categorical')

Found 1281167 images belonging to 1000 classes.
Found 50000 images belonging to 1000 classes.


In [4]:
img_w = 224
img_h = 224
img_c = 3

inp = Input(shape=(img_w, img_h, img_c))

model = MobileNet(input_tensor=inp, shallow=True)

In [5]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 32)      864       
_________________________________________________________________
batch_normalization_1 (Batch (None, 112, 112, 32)      128       
_________________________________________________________________
activation_1 (Activation)    (None, 112, 112, 32)      0         
_________________________________________________________________
depthwise_conv2d_1 (Depthwis (None, 112, 112, 32)      288       
_________________________________________________________________
batch_normalization_2 (Batch (None, 112, 112, 32)      128       
_________________________________________________________________
activation_2 (Activation)    (None, 112, 112, 32)      0         
__________

In [7]:
tb_callback = TB_writer(histogram_freq=5, write_images=True, log_dir="mobilenet", val_gen=validation_generator)
tb_callback.set_model(model)

In [8]:
model.fit_generator(
        train_generator,
        steps_per_epoch = train_generator.samples / train_generator.batch_size,
        epochs = 10,
        validation_data = validation_generator,
        validation_steps = validation_generator.samples / validation_generator.batch_size,
        callbacks=[tb_callback])

Epoch 1/10
   62/10009 [..............................] - ETA: 13026s - loss: 7.1990 - acc: 0.0010 

/usr/local/lib/python2.7/dist-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1835008 bytes but only got 0. Skipping tag 0
  "Skipping tag %s" % (size, len(data), tag))


  465/10009 [>.............................] - ETA: 12387s - loss: 6.6446 - acc: 0.0071

/usr/local/lib/python2.7/dist-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2555904 bytes but only got 0. Skipping tag 0
  "Skipping tag %s" % (size, len(data), tag))


 3216/10009 [========>.....................] - ETA: 9595s - loss: 5.2015 - acc: 0.0829

/usr/local/lib/python2.7/dist-packages/PIL/TiffImagePlugin.py:709: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


 9534/10009 [===========================>..] - ETA: 627s - loss: 4.1026 - acc: 0.2046

/usr/local/lib/python2.7/dist-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 19660800 bytes but only got 0. Skipping tag 0
  "Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python2.7/dist-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 18481152 bytes but only got 0. Skipping tag 0
  "Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python2.7/dist-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 37093376 bytes but only got 0. Skipping tag 0
  "Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python2.7/dist-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 39976960 bytes but only got 0. Skipping tag 0
  "Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python2.7/dist-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to re

10009/10009 [==============================] - 13492s - loss: 4.0552 - acc: 0.2108 - val_loss: 3.2710 - val_acc: 0.3067
Epoch 2/10
10009/10009 [==============================] - 13001s - loss: 2.7248 - acc: 0.4005 - val_loss: 3.0465 - val_acc: 0.3555
Epoch 3/10
10009/10009 [==============================] - 14008s - loss: 2.3196 - acc: 0.4743 - val_loss: 2.5442 - val_acc: 0.4363
Epoch 4/10
10009/10009 [==============================] - 14015s - loss: 2.0867 - acc: 0.5180 - val_loss: 2.6439 - val_acc: 0.4244
Epoch 5/10
10009/10009 [==============================] - 13032s - loss: 1.9279 - acc: 0.5491 - val_loss: 2.2872 - val_acc: 0.4872
Epoch 6/10
10009/10009 [==============================] - 13124s - loss: 1.8093 - acc: 0.5723 - val_loss: 2.1654 - val_acc: 0.5108
Epoch 7/10
10009/10009 [==============================] - 13099s - loss: 1.7163 - acc: 0.5909 - val_loss: 2.2054 - val_acc: 0.5037
Epoch 8/10
10009/10009 [==============================] - 13078s - loss: 1.6384 - acc: 0.6064 

In [9]:
train_generator.samples

1281167

In [10]:
model.save("mobilenet.h5")